In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

# load in sc data

In [2]:
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')
adata_1 = adata_T[adata_T[:,'PDCD1'].X > 0, :]
adata_Endo = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Endo_cells.h5ad')
adata_2 = adata_Endo

In [4]:
colors = ["grey", "blue"]  # Start with grey and end with blue
cmap = mcolors.LinearSegmentedColormap.from_list("grey_to_blue", colors)


In [5]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_2, sample_id_col='sample_id')

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


# prepare data matrix for IVtest

In [6]:
DEG_1 = paird_ttest(adata_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = paird_ttest(adata_2, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
DEG_1 = DEG_1[DEG_1['pval'] < 0.05]
DEG_2 = DEG_2[DEG_2['pval'] < 0.05]
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Endo' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')

In [8]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

0    31
1    30
Name: treatment, dtype: int64

## group the DEGpairs together using L and R

In [22]:
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1vsEndo_fisherz_corr03.xlsx')
# results = results.drop(columns=['Unnamed: 0'])
# # Remove the suffix '_X' from g1 and g2
results['g1'] = results['g1'].str.replace(r'_\w+', '', regex=True)
results['g2'] = results['g2'].str.replace(r'_\w+', '', regex=True)

In [23]:
# only keep the significant results with correlation > 0.3 or < -0.3
results = results[(results['g1vsg2_correlation'] > 0.3)]

In [24]:
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,0,IRF1,IL6R,ICAM1,CSF2RB,0.307114,0.542384,0.141500
1,1,CXCR4,PHLDA1,ANXA1,F2RL3,0.054875,0.529491,0.247787
2,2,IL7R,MT1M,ANXA1,AGTR1,0.064600,0.520054,0.247954
3,3,CCNH,F2RL3,ANXA1,F2RL3,0.517159,0.503231,0.080566
4,4,FASLG,ZNF383,FASLG,FAS,0.115643,0.498808,0.201678
...,...,...,...,...,...,...,...,...
1508,1508,AC025164.1,TSC22D3,APP,EDNRB,0.055134,0.300768,0.230172
1509,1509,AC025164.1,TSC22D3,APP,F2RL3,0.113279,0.300768,0.189279
1510,1510,AC025164.1,TSC22D3,CCL5,CXCR4,0.050405,0.300768,0.266891
1511,1511,AC025164.1,TSC22D3,PECAM1,ACKR3,0.050679,0.300768,0.231202


In [25]:
calculate_g1_L_correlation(results, adata_1_pseudo)
calculate_g2_R_correlation(results, adata_2_pseudo)

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
0,0,IRF1,IL6R,ICAM1,CSF2RB,0.307114,0.542384,0.141500,0.554250,0.211990
1,1,CXCR4,PHLDA1,ANXA1,F2RL3,0.054875,0.529491,0.247787,0.506629,0.395634
2,2,IL7R,MT1M,ANXA1,AGTR1,0.064600,0.520054,0.247954,0.486636,0.655233
3,3,CCNH,F2RL3,ANXA1,F2RL3,0.517159,0.503231,0.080566,0.186274,1.000000
4,4,FASLG,ZNF383,FASLG,FAS,0.115643,0.498808,0.201678,1.000000,0.446194
...,...,...,...,...,...,...,...,...,...,...
1508,1508,AC025164.1,TSC22D3,APP,EDNRB,0.055134,0.300768,0.230172,0.224024,0.438735
1509,1509,AC025164.1,TSC22D3,APP,F2RL3,0.113279,0.300768,0.189279,0.224024,0.461363
1510,1510,AC025164.1,TSC22D3,CCL5,CXCR4,0.050405,0.300768,0.266891,-0.251768,-0.186414
1511,1511,AC025164.1,TSC22D3,PECAM1,ACKR3,0.050679,0.300768,0.231202,0.221842,0.145343


In [26]:
# remove the rows with g1vsL_correlation < 0.3 
results = results[abs((results['g1vsL_correlation'] > 0.3)) & (results['g1vsL_correlation'] < 0.99)]
# remove the rows with g2vsR_correlation < 0.3 
results = results[abs((results['g2vsR_correlation'] > 0.3)) & (results['g2vsR_correlation'] < 0.99)]

In [27]:
# get the unique (L,R) pairs in results
lr_pairs = results[['L', 'R']]
# remove the LR pairs repeated less than 3 times in the lr_pairs
lr_pairs = lr_pairs.groupby(['L', 'R']).filter(lambda x: len(x) >= 2)
lr_pairs = lr_pairs.drop_duplicates()
lr_pairs

,L,R
1,ANXA1,F2RL3
2,ANXA1,AGTR1
11,PTPRF,RET
15,ANXA1,EDNRB
21,AREG,ICAM1
27,ICAM1,CSF2RB
28,ANXA1,LIFR
29,ANXA1,TACR1
30,ANXA1,MET
35,ANXA1,EGFR


In [28]:
import pandas as pd
grouped_results = results.groupby(["L", "R"])

# Display grouped data
grouped_dict = { (ligand, receptor): group for (ligand, receptor), group in grouped_results }

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
grouped_data = []
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1 and GEM_g2
    # and where the overlap is below the maximum allowable threshold
    if len(gem1) > min_gene_num and len(gem2) > min_gene_num:
        temp_data.append({
            "L": ligand,
            "R": receptor,
            "g1": gem1,
            "g2": gem2,
            "Num_genes_g1": len(gem1),
            "Num_genes_g2": len(gem2)
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)

# Function to find overlapping rows
def find_overlapping_rows(df, column_name):
    overlap_info = []
    for index, row in df.iterrows():
        current_set = set(row[column_name])
        overlapping_rows = []
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_set = set(other_row[column_name])
                overlap = len(current_set & other_set)
                if overlap > len(current_set) / 3: # Set the threshold for overlap
                    overlapping_rows.append(other_index)
        overlap_info.append(overlapping_rows)
    return overlap_info

# Add columns for overlapping rows in GEM_g1 and GEM_g2
grouped_df["Overlapping_g1"] = find_overlapping_rows(grouped_df, "g1")
grouped_df["Overlapping_g2"] = find_overlapping_rows(grouped_df, "g2")


In [29]:
# rank each GEM_g1 by order of name inside
grouped_df['g1'] = grouped_df['g1'].apply(lambda x: sorted(x))
grouped_df['g2'] = grouped_df['g2'].apply(lambda x: sorted(x))
grouped_df

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2
0,ANXA1,AGTR1,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[CDKN1A, CREB5, ERRFI1, FAM107A, FKBP5, INHBB,...",7,10,"[1, 2, 3, 4]","[2, 7, 10, 11]"
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADM, BHLHE40, CDKN1A, CREB5, CSRNP1, F2RL3, F...",6,15,"[0, 2, 3, 4]","[3, 12]"
2,ANXA1,EGFR,"[SESN1, TXNIP, ZFP36L2]","[ERRFI1, FAM107A, FKBP5, INHBB]",3,4,"[0, 1, 3, 4, 10, 12]","[0, 4, 7, 10, 11]"
3,ANXA1,F2RL3,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[ADM, BHLHE40, CDKN1A, CREB5, ERRFI1, HES1, IT...",7,13,"[0, 1, 2, 4]","[1, 12, 13]"
4,ANXA1,LIFR,"[CITED2, CXCR4, GADD45B, IL7R, SESN1, TNFSF14,...","[BBS9, CLU, ERRFI1, FAM107A, IER2, INHBB, NFKBIA]",7,7,"[0, 1, 3, 5, 6]","[0, 2, 5, 6, 10, 11]"
5,ANXA1,MET,"[CXCR4, GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",4,3,"[0, 1, 3, 4, 6]","[4, 6]"
6,ANXA1,TACR1,"[GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",3,3,"[4, 5]","[4, 5]"
7,AREG,EGFR,"[CBLB, CCNH, CD55, CEMIP2, CITED2, PARP8, SRGN...","[CEBPD, CSRNP1, DUSP1, EGR1, ERRFI1, FAM107A, ...",9,12,[],[0]
8,AREG,ICAM1,"[CRYBG1, NFKBIA, SRGN, TAGAP, TNFAIP3, TSC22D3...","[ATF3, NFKBIA, NFKBIZ, SLC2A3]",7,4,[10],[]
9,FASLG,FAS,"[AC004687.1, KLF10, TNFSF14]","[AC116407.2, BBS9, ID1, ID3, PIM3, RNPC3]",3,6,[],[]


In [30]:
# remove _0 and _1 in index of adata_2_pseudo
adata_2_pseudo.obs.index = adata_2_pseudo.obs.index.str.replace('-0', '').str.replace('-1', '')
adata_1_pseudo.obs.index = adata_1_pseudo.obs.index.str.replace('-0', '').str.replace('-1', '')

In [31]:
#calculate the correlation between g1 and g2
calculate_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)
#calculate the residuals correlation between g1, g2 and LR
calculate_residuals_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,ANXA1,AGTR1,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[CDKN1A, CREB5, ERRFI1, FAM107A, FKBP5, INHBB,...",7,10,"[1, 2, 3, 4]","[2, 7, 10, 11]",0.678096,0.539951
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADM, BHLHE40, CDKN1A, CREB5, CSRNP1, F2RL3, F...",6,15,"[0, 2, 3, 4]","[3, 12]",0.668203,0.541821
2,ANXA1,EGFR,"[SESN1, TXNIP, ZFP36L2]","[ERRFI1, FAM107A, FKBP5, INHBB]",3,4,"[0, 1, 3, 4, 10, 12]","[0, 4, 7, 10, 11]",0.633708,0.525352
3,ANXA1,F2RL3,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[ADM, BHLHE40, CDKN1A, CREB5, ERRFI1, HES1, IT...",7,13,"[0, 1, 2, 4]","[1, 12, 13]",0.657121,0.468941
4,ANXA1,LIFR,"[CITED2, CXCR4, GADD45B, IL7R, SESN1, TNFSF14,...","[BBS9, CLU, ERRFI1, FAM107A, IER2, INHBB, NFKBIA]",7,7,"[0, 1, 3, 5, 6]","[0, 2, 5, 6, 10, 11]",0.529098,0.198198
5,ANXA1,MET,"[CXCR4, GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",4,3,"[0, 1, 3, 4, 6]","[4, 6]",0.400902,0.026151
6,ANXA1,TACR1,"[GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",3,3,"[4, 5]","[4, 5]",0.471197,0.163114
7,AREG,EGFR,"[CBLB, CCNH, CD55, CEMIP2, CITED2, PARP8, SRGN...","[CEBPD, CSRNP1, DUSP1, EGR1, ERRFI1, FAM107A, ...",9,12,[],[0],0.733957,0.635754
8,AREG,ICAM1,"[CRYBG1, NFKBIA, SRGN, TAGAP, TNFAIP3, TSC22D3...","[ATF3, NFKBIA, NFKBIZ, SLC2A3]",7,4,[10],[],0.543477,0.370630
9,FASLG,FAS,"[AC004687.1, KLF10, TNFSF14]","[AC116407.2, BBS9, ID1, ID3, PIM3, RNPC3]",3,6,[],[],0.584406,0.434008


In [35]:
grouped_df['GEM_1'] = 'GEM_' + grouped_df.index.astype(str) + '_T'
grouped_df['GEM_2'] = 'GEM_' + grouped_df.index.astype(str) + '_Endo'

In [36]:
# Define a function to append '_T' to each gene name
def append_suffix(gene_list, suffix='_T'):
    # Ensure it's a list and not NaN or another unexpected type
    if isinstance(gene_list, list):
        return [gene + suffix for gene in gene_list]
    else:
        print("Unexpected format:", gene_list)
        return gene_list

# Apply the function to the GEM_g1 column to add '_T' to each gene name
DEG_pairs_T = grouped_df.copy()
DEG_pairs_T['g1'] = grouped_df['g1'].apply(lambda x: append_suffix(x, '_T'))

# Similarly for GEM_g2 if needed
DEG_pairs_T['g2'] = grouped_df['g2'].apply(lambda x: append_suffix(x, '_Endo'))
# remove the genes not in gene_df from each row in DEG_pairs_T but keep the row
DEG_pairs_T['g1'] = DEG_pairs_T['g1'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])
DEG_pairs_T['g2'] = DEG_pairs_T['g2'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])


# add GEM_1_T, GEM_2_T and so on to gene_df
for i, row in DEG_pairs_T.iterrows():
    GEM_1 = DEG_pairs_T.loc[i, 'g1']
    GEM_2 = DEG_pairs_T.loc[i, 'g2']
    gene_df['GEM_' + str(i) + '_T'] = gene_df[GEM_1].mean(axis=1)
    gene_df['GEM_' + str(i) + '_Endo'] = gene_df[GEM_2].mean(axis=1)
gene_df

,AC004687.1_T,AC004817.3_T,AC004951.1_T,AC006369.1_T,AC007038.2_T,AC015813.1_T,AC016831.5_T,AC021188.1_T,AC023157.3_T,AC025164.1_T,...,GEM_4_Endo,GEM_5_Endo,GEM_6_Endo,GEM_7_Endo,GEM_8_Endo,GEM_9_Endo,GEM_10_Endo,GEM_11_Endo,GEM_12_Endo,GEM_13_Endo
sample_id,,,,,,,,,,,,,,,,,,,,,
BIOKEY_13_Pre,496.245117,71.430817,0.000000,221.554840,0.000000,0.000000,0.000000,50.375000,66.970314,190.125916,...,926.618591,1815.373413,1815.373413,520.773010,908.894775,1521.873413,510.767914,543.238159,548.756409,399.205719
BIOKEY_13_On,548.931458,73.001106,105.453827,215.637390,50.366783,29.903372,0.000000,249.275970,106.639206,474.283173,...,1083.501709,1779.076538,1779.076538,1025.638306,957.647400,1178.225952,785.407288,811.967346,770.590027,659.108032
BIOKEY_10_Pre,563.980347,44.845234,55.078514,81.338425,5.761608,25.467459,9.838195,82.176804,36.126411,164.164597,...,676.052246,1330.172974,1330.172974,277.535614,593.745178,1164.647461,438.534912,327.076996,409.472229,367.938660
BIOKEY_10_On,309.215912,189.972443,46.985409,88.311661,13.239152,40.373478,43.434666,111.079369,77.687202,379.605682,...,682.496887,1211.854248,1211.854248,721.295227,822.398438,971.732117,537.105286,427.947357,944.880981,798.745544
BIOKEY_16_Pre,484.466187,89.122238,27.256454,405.647430,4.384324,32.537617,47.155140,20.080681,12.754025,204.002930,...,665.026001,1103.403809,1103.403809,619.512146,681.655579,1189.982056,590.305237,487.046997,790.232849,603.097656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIOKEY_17_On,348.290375,523.852295,0.000000,0.000000,88.987106,0.000000,0.000000,165.759293,252.315216,456.512634,...,669.264954,1244.140747,1244.140747,808.985840,965.538513,954.262207,574.537598,518.973816,1100.322754,858.116516
BIOKEY_27_Pre,724.247070,0.000000,0.000000,234.127594,0.000000,0.000000,60.133003,0.000000,0.000000,68.258476,...,898.420776,1665.026978,1665.026978,502.652252,719.274292,1301.124512,438.478668,442.526978,502.946991,403.004639
BIOKEY_27_On,54.874050,330.519226,53.683594,225.546738,0.000000,58.787476,182.094040,169.446136,245.524399,598.750000,...,1024.882568,1654.533203,1654.533203,1373.384399,1226.136841,877.519470,837.609375,888.312683,1034.107788,889.506897


In [37]:
# add GEM_1_T, GEM_2_T and so on to adata
DEG_pairs = grouped_df
for i, row in DEG_pairs.iterrows():
    GEM_1 = DEG_pairs.loc[i, 'g1']
    GEM_2 = DEG_pairs.loc[i, 'g2']
    # add GEMs to adata_1_pseudo and adata_2_pseudo
    adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
    adata_2_pseudo.obs['GEM_' + str(i) + '_Endo'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
    # add GEMs to adata_1 and adata_2
    adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)
    adata_2.obs['GEM_' + str(i) + '_Endo'] = adata_2[:, GEM_2].X.mean(axis=1)

/tmp/ipykernel_1577614/2359226253.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
/tmp/ipykernel_1577614/2359226253.py:8: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_2_pseudo.obs['GEM_' + str(i) + '_Endo'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
/tmp/ipykernel_1577614/2359226253.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)


In [38]:
CIT_GEM_test(grouped_df, adata_1_pseudo, adata_2_pseudo, gene_df, method="fisherz")

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,GEM_1,GEM_2,pValue
0,ANXA1,AGTR1,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[CDKN1A, CREB5, ERRFI1, FAM107A, FKBP5, INHBB,...",7,10,"[1, 2, 3, 4]","[2, 7, 10, 11]",0.678096,0.539951,GEM_0_T,GEM_0_Endo,4.052269e-06
1,ANXA1,EDNRB,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36L2]","[ADM, BHLHE40, CDKN1A, CREB5, CSRNP1, F2RL3, F...",6,15,"[0, 2, 3, 4]","[3, 12]",0.668203,0.541821,GEM_1_T,GEM_1_Endo,6.329909e-07
2,ANXA1,EGFR,"[SESN1, TXNIP, ZFP36L2]","[ERRFI1, FAM107A, FKBP5, INHBB]",3,4,"[0, 1, 3, 4, 10, 12]","[0, 4, 7, 10, 11]",0.633708,0.525352,GEM_2_T,GEM_2_Endo,5.672789e-06
3,ANXA1,F2RL3,"[CITED2, CXCR4, IL7R, SESN1, TXNIP, ZFP36, ZFP...","[ADM, BHLHE40, CDKN1A, CREB5, ERRFI1, HES1, IT...",7,13,"[0, 1, 2, 4]","[1, 12, 13]",0.657121,0.468941,GEM_3_T,GEM_3_Endo,1.535310e-05
4,ANXA1,LIFR,"[CITED2, CXCR4, GADD45B, IL7R, SESN1, TNFSF14,...","[BBS9, CLU, ERRFI1, FAM107A, IER2, INHBB, NFKBIA]",7,7,"[0, 1, 3, 5, 6]","[0, 2, 5, 6, 10, 11]",0.529098,0.198198,GEM_4_T,GEM_4_Endo,1.308844e-01
5,ANXA1,MET,"[CXCR4, GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",4,3,"[0, 1, 3, 4, 6]","[4, 6]",0.400902,0.026151,GEM_5_T,GEM_5_Endo,8.816020e-01
6,ANXA1,TACR1,"[GADD45B, IL7R, TNFSF14]","[CLU, IER2, NFKBIA]",3,3,"[4, 5]","[4, 5]",0.471197,0.163114,GEM_6_T,GEM_6_Endo,7.142290e-01
7,AREG,EGFR,"[CBLB, CCNH, CD55, CEMIP2, CITED2, PARP8, SRGN...","[CEBPD, CSRNP1, DUSP1, EGR1, ERRFI1, FAM107A, ...",9,12,[],[0],0.733957,0.635754,GEM_7_T,GEM_7_Endo,2.602822e-10
8,AREG,ICAM1,"[CRYBG1, NFKBIA, SRGN, TAGAP, TNFAIP3, TSC22D3...","[ATF3, NFKBIA, NFKBIZ, SLC2A3]",7,4,[10],[],0.543477,0.370630,GEM_8_T,GEM_8_Endo,3.249395e-04
9,FASLG,FAS,"[AC004687.1, KLF10, TNFSF14]","[AC116407.2, BBS9, ID1, ID3, PIM3, RNPC3]",3,6,[],[],0.584406,0.434008,GEM_9_T,GEM_9_Endo,5.716097e-05


In [39]:
# save the grouped_df to excel
grouped_df.to_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1vsEndo_grouped_0920.xlsx')